## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
from IPython.display import display 
display(data.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Дана таблица, состоящая из 12 столбцов и 21525 строк. В 2 столбцах есть пропуски в данных. По типу данных 2 столбца вещественные числа, 5 столбцов целочисленные, 5 столбцов тип данных строка.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#print(data.isna().sum())

data.loc[(data['days_employed'].isna()) & (data['dob_years']<=65),['days_employed']] = ((data['dob_years'] - 18) * 247) #расчет теоретического кол-ва трудодней 
data.loc[(data['days_employed'].isna()) & (data['dob_years']>65),['days_employed']] = 11609 #расчет теоретического кол-ва трудодней за 47 лет(65л - 18л) 

data['education'] = data['education'].str.lower()
group_education = data.groupby('education')['total_income'].median()
print(group_education)

data.loc[(data['total_income'].isna()) & (data['education']=='среднее'),['total_income']] = 136478.643244
data.loc[(data['total_income'].isna()) & (data['education']=='ученая степень'),['total_income']] = 157259.898555
data.loc[(data['total_income'].isna()) & (data['education']=='неоконченное высшее'),['total_income']] =160115.398644
data.loc[(data['total_income'].isna()) & (data['education']=='начальное'),['total_income']] = 117137.352825
data.loc[(data['total_income'].isna()) & (data['education']=='высшее'),['total_income']] = 175340.818855


#data.info()
display(data.head(20))
print(data[data['days_employed'].isna()==True])

education
высшее                 175340.818855
начальное              117137.352825
неоконченное высшее    160115.398644
среднее                136478.643244
ученая степень         157259.898555
Name: total_income, dtype: float64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


### Вывод   

Обнаружены пропуски в 2х столбцах: 'days_employed' и 'total_income'. Возможные причины пропусков - отсутствие информации от клиента(например отсутствует официальный стаж работы, нет подтвержденного дохода). Пропуски столбца 'days_employed' заполняем теоретическим значением трудодней, расчитанных исходя из возраста клиента и среднего количества рабочих дней в году(247 дней). Клиентам старше 65 лет(пенсионерам) пропуски заменям на теоретически максимальное количество трудодней. Замена пропусков производится через индексацию.
При выводе таблицы сразу бросается в глаза колонка 'education' с разным регистром. Приводим всю колонку к нижнему регистру, тем самым избавляемся от дубликатов. Для столбца 'total_income' пропуски заполняем средними значениями зарплат исходя из типа образования(высчитываем медиану зарплат по каждому типу образования методом median()). Далее с помощью индексации заполняем пропуски.

### Замена типа данных

In [3]:
data.loc[data['days_employed'] > 11609,['days_employed']] = ((data['dob_years'] - 18) * 247)
#print(data.tail(30))

data['days_employed'] = abs(data['days_employed'])
data['days_employed'] = data['days_employed'].astype('int')

 
display(data.tail(50))
data.info()



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21475,0,8398,52,среднее,1,женат / замужем,0,F,пенсионер,1,48239.310903,свой автомобиль
21476,0,2539,52,высшее,0,женат / замужем,0,M,компаньон,0,527452.833253,операции со своей недвижимостью
21477,0,1628,54,среднее,1,гражданский брак,1,F,сотрудник,0,71733.906470,на проведение свадьбы
21478,1,921,28,высшее,0,женат / замужем,0,F,сотрудник,0,181185.864790,свой автомобиль
21479,0,685,26,среднее,1,женат / замужем,0,M,сотрудник,0,171504.352534,покупка жилья для сдачи
21480,0,8799,45,высшее,0,Не женат / не замужем,4,F,сотрудник,0,146945.045428,ремонт жилью
21481,0,829,30,неоконченное высшее,2,гражданский брак,1,M,сотрудник,0,440470.880955,на покупку автомобиля
21482,1,865,25,среднее,1,женат / замужем,0,F,компаньон,0,94065.031245,свой автомобиль
21483,2,2917,38,начальное,3,гражданский брак,1,F,сотрудник,1,77417.560520,операции с недвижимостью
21484,3,968,35,среднее,1,женат / замужем,0,M,сотрудник,0,95667.452395,жилье


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод данные о стаже работы изменены на целочисленный тип данных

В столбце 'days_employed' тип данных float64, трудодни  - это целочисленный показатель и следовательно нужно изменить тип данных на int64. Рассмотрим детально данные.Присутствуют 3 вида: 1. Предположительно верные; 2. Отрицательные значения (возможно привводе данных или выгрузке произошла ошибка и к данным присоединился символ "-"; 3. Слишком большие значения(даже если брать стаж работы с 18 лет до 65 лет при среднем количестве рабочих дней в году равном 247, то количество трудодней должно быть не более 11609). Прежде чем изменить тип данных столбца, нужно обработать сомнительные показатели. Поскольку процент слишком больших значений очень мал, мы избавляемся от этих данных, заменяя на нулевое значение. Далее пишем код для "превращения" отрицательных значений в положительные. После обработки данных можно изменить тип данных на целочисленные. В данном случае подходит метод astype() с заменой старого столбца на новый.

### Обработка дубликатов

In [4]:
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())


71
0


### Вывод

Проверяем таблицу на наличие дубликатов. Для дубликатов в данной ситуации подходит метод  data.duplicated().sum(). Далее удаляем дубликаты и обновляем индексы методом drop_duplicates().reset_index()

### Лемматизация

In [5]:
purpose_credit = data['purpose']
from pymystem3 import Mystem
m = Mystem()
lemmas=[]
for i in purpose_credit:
    element = m.lemmatize(i)
    lemmas.append(element)
data['purpose'] = lemmas
print(lemmas)


[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['покупка', ' ', 'жилье', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['недвижимость', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'подержа

### Вывод

Для лемматизации выбран столбец с кредитными целями. Для поиска ключевых слов(целей) выбрана библиотека на русском языке pymystem3.Создаем пустой datafram и заполняем путем лемматизации(m.lemmatize()) каждого эелемента столбца с помощью цикла.
Далее заменяем старый столбец 'purpose' на новый лемматизированный lemmas.


### Категоризация данных

In [6]:
def key_purpose(data):   
    element = data['purpose']
    if 'недвижимость' in element  or 'жилье' in element or 'жилой'in element:
        return 'недвижимость'            
    if 'автомобиль' in element:
        return 'автомобиль'            
    if 'образование' in element:
        return 'образование'            
    if 'свадьба' in element:
        return 'свадьба'            
    if 'ремонт' in element:
        return 'ремонт жилья'            
    
data['purpose'] = data.apply(key_purpose, axis=1)   


display(data.head(10))



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,образование
4,0,8645,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,недвижимость


### Вывод

Для категорицации выбран столбец с целями на кредит. После лемматизации в столбце 'purpose' записаны цели в словарной форме и определены 5 категорий(недвижимость, автомобиль, образование, свадьба, ремонт). Категоризуем цели путем создания функции и вызова метода apply(). В функции проверяем каждую строку столбца 'purpose' на соответствие словарным формам  и ообъединяем в категории. 
Метод apply() позволяет получить готовый столбец с группой. Поскольку в функции мы работаем со строками, то указываем пераметр axis = 1 для метода apply().

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
children_id = []
for i in data['children']:
    if i != 0:
        id = 'есть дети' # есть дети
    else:
        id = 'нет детей' # нет детей
    children_id.append(id) 
data['children_id'] = children_id #идентификатор наличия детей
client_id = []
number = 0
for i in data['children_id']:
    number += 1
    client_id.append(number)
data['client_id'] = client_id  #идентификатор клиента


# Сводная таблица

children_pivot = data.pivot_table(index=['children_id', 'debt'],  aggfunc={'client_id':'count'})
total_debt_children = children_pivot.groupby('children_id')['client_id'].sum() #общее количество должников в данной категории
children_pivot['debt_share'] = children_pivot['client_id'] / total_debt_children
display(children_pivot)

client_id  debt_share
children_id debt                       
есть дети   0          6685    0.907918
            1           678    0.092082
нет детей   0         13028    0.924562
            1          1063    0.075438

### Вывод

При анализе таблицы сравниваем доли возвратов кредитов в срок и доли просроченных кредитов. Для этого сформируем сводную таблицу, в которой будут отображаться данные о наличии детей, долгов, количество клиентов в каждой категории и доли должников из расчета общем числа должников в категории. В результате изучения полученной таблицы можно дать ответ на поставленный вопрос: доля просроченных кредитов при наличии детей у заемщиков больше, чем доля у клиентов без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
family_pivot = data.pivot_table(index=['family_status', 'debt'], aggfunc={'client_id':'count'})
total_debt_family = family_pivot.groupby('family_status')['client_id'].sum()#общее количество должников в данной категории
family_pivot['debt_share'] = family_pivot['client_id'] / total_debt_family
display(family_pivot)



client_id  debt_share
family_status         debt                       
Не женат / не замужем 0          2536    0.902491
                      1           274    0.097509
в разводе             0          1110    0.928870
                      1            85    0.071130
вдовец / вдова        0           896    0.934307
                      1            63    0.065693
гражданский брак      0          3763    0.906529
                      1           388    0.093471
женат / замужем       0         11408    0.924548
                      1           931    0.075452

### Вывод

Для анализа влияния семейного положения на своевременную выплату кредита формируем сводную таблицу с данными по семейному положению, долгу по кредиту, количеству клиентов, долей должников по категориям. В результате исслендования можно сделать вывод, что зависимость между семейным положением и возвратом кредита в срок есть(например заемщики в гражданском чаще просрочивают выплаты, нежели вдовцы).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
#категоризация по доходу
def income_category(income):
    if income < 100000:
        return 'низкий'
    if income <= 300000:
        return 'средний'
    if income <= 1000000:
        return 'выше среднего'
    return 'высокий' 

data['income_group'] = data['total_income'].apply(income_category)
#display(data. head(10))          


income_pivot = data.pivot_table(index=['income_group', 'debt'], aggfunc={'client_id':'count'})
total_debt_income = income_pivot.groupby('income_group')['client_id'].sum()#общее количество должников в данной категории
income_pivot['debt_share'] = income_pivot['client_id'] / total_debt_income
display(income_pivot)


client_id  debt_share
income_group  debt                       
высокий       0            23    0.920000
              1             2    0.080000
выше среднего 0          1353    0.928620
              1           104    0.071380
низкий        0          4109    0.920681
              1           354    0.079319
средний       0         14228    0.917403
              1          1281    0.082597

### Вывод

Для ответа на поставленный вопрос нам понадобится сводная таблица с категориями дохода, данными по долгам, количеством клиентов и долей должников по каждой категории. В результате полученной таблицы можно судить о том, что четкой тендеции влияния дохода на своевремнное погашение кредита не прослеживается.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
purpose_pivot = data.pivot_table(index=['purpose', 'debt'], aggfunc={'client_id':'count'})
total_debt_purpose = purpose_pivot.groupby('purpose')['client_id'].sum()#общее количество должников в данной категории
purpose_pivot['debt_share'] = purpose_pivot['client_id'] / total_debt_purpose
display(purpose_pivot)



client_id  debt_share
purpose      debt                       
автомобиль   0          3903    0.906410
             1           403    0.093590
недвижимость 0         10029    0.927666
             1           782    0.072334
образование  0          3643    0.907800
             1           370    0.092200
свадьба      0          2138    0.919966
             1           186    0.080034

### Вывод

Для анализа влияния цели на своевременную выплату кредита была сформирована сводная таблица, состоящая из целевых категорий, данных о долгах, количества клиентов и долей должников по каждой категории. Изучая полученную таблицу можно отметить, что например на категорию "автомобиль" приходится большая доля просрочек по кредитам, а на "недвижимость" наименьшая.

### Шаг 4. Общий вывод

Врезультате работы над данным проектом данные прошли предобработку и смогли дать ответы на поставленные заказчиком вопросы. В частности выяснили какие факторы  могут влиять на своевременное погашение кредитов:
1. Процент просроченных кредитов при наличии детей у заемщиков оказался больше(9,2%), чем у клиентов без детей(7,5%).
2. Есть зависимость между семейным положением и возвратом кредита в срок. 
    Количество просроченных кредитов:
    Не женат / не замужем   9,75%
    гражданский брак        9,35%
    женат / замужем         7,55%
    в разводе               7,11%
    вдовец / вдова          6,57%
3. Четкой тендеции влияния дохода на своевремнное погашение кредита не выявлено.
   Показатели просроченных кредитов согласно доходу:
    низкий        7,9%
    средний       8,2%
    выше среднего 7,1%
    высокий       8,0%    
4. Кредитные цели влияют на несвоевременную выплату:
    автомобиль    9,4%
    образование   9,2%
    свадьба       8,0%
    недвижимость  7,2%
    
    
